### Download PDB structures corresponding to GPCR Uniprot

In [ ]:
import time
from pathlib import Path
import pandas as pd
import requests
from tqdm.auto import tqdm

IN_CSV = Path("GPCR_Human_PDB_info.csv")
OUT_CSV = Path("GPCR_Uniprot2PDB.csv")

# UniProt Accession column
ACC_COL = "Entry"  

SEARCH_URL = "https://search.rcsb.org/rcsbsearch/v2/query"
CORE_ENTRY_URL = "https://data.rcsb.org/rest/v1/core/entry/{}"

SLEEP_SEC = 0.05
RETRIES = 5
TIMEOUT = 60

def _sleep_backoff(attempt):
    time.sleep((2 ** attempt) * 0.5)

def post_json(session, url, payload, retries=RETRIES, timeout=TIMEOUT):
    last_err = None
    for attempt in range(retries):
        try:
            r = session.post(url, json=payload, timeout=timeout)
            if r.status_code in (429, 502, 503, 504):
                _sleep_backoff(attempt)
                continue
            r.raise_for_status()
            return r.json()
        except Exception as e:
            last_err = e
            _sleep_backoff(attempt)
    raise last_err

def get_json(session, url, retries=RETRIES, timeout=TIMEOUT):
    last_err = None
    for attempt in range(retries):
        try:
            r = session.get(url, timeout=timeout)
            if r.status_code in (429, 502, 503, 504):
                _sleep_backoff(attempt)
                continue
            r.raise_for_status()
            return r.json()
        except Exception as e:
            last_err = e
            _sleep_backoff(attempt)
    raise last_err

def rcsb_pdb_ids_by_uniprot(session, accession):
    acc = str(accession).strip().upper()
    if not acc or acc == "NAN":
        return []

    payload = {
        "query": {
            "type": "group",
            "logical_operator": "and",
            "nodes": [
                {
                    "type": "terminal",
                    "service": "text",
                    "parameters": {
                        "attribute": "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_name",
                        "operator": "exact_match",
                        "value": "UniProt",
                    },
                },
                {
                    "type": "terminal",
                    "service": "text",
                    "parameters": {
                        "attribute": "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
                        "operator": "exact_match",
                        "value": acc,
                    },
                },
            ],
        },
        "return_type": "entry",
        "request_options": {
            "return_all_hits": True,
            "results_content_type": ["experimental"],
        },
    }

    j = post_json(session, SEARCH_URL, payload)
    hits = j.get("result_set", []) or []
    return sorted({h.get("identifier") for h in hits if h.get("identifier")})

def extract_pdbx_database_related_details(entry_json):
    items = entry_json.get("pdbx_database_related", []) or []
    details_list = []
    for it in items:
        if isinstance(it, dict):
            d = it.get("details")
            if d:
                details_list.append(str(d).strip())
    return " | ".join(details_list) if details_list else None


df = pd.read_csv(IN_CSV, dtype={ACC_COL: "string"})

accessions = df[ACC_COL].dropna().astype(str).str.strip().str.upper()
accessions = accessions[accessions.ne("") & accessions.ne("NAN")].unique().tolist()

session = requests.Session()
session.headers.update({"User-Agent": "GPCRbias/rcsb-search+core"})

acc_cache = {}
pdb_details_cache = {}

rows = []
for acc in tqdm(accessions, desc="UniProt -> PDB", unit="acc"):
    if acc in acc_cache:
        pdbs = acc_cache[acc]
    else:
        try:
            pdbs = rcsb_pdb_ids_by_uniprot(session, acc)
        except Exception:
            pdbs = []
        acc_cache[acc] = pdbs
        time.sleep(SLEEP_SEC)

    for pdb in pdbs:
        if pdb in pdb_details_cache:
            details = pdb_details_cache[pdb]
        else:
            try:
                entry = get_json(session, CORE_ENTRY_URL.format(pdb))
                details = extract_pdbx_database_related_details(entry)
            except Exception:
                details = None
            pdb_details_cache[pdb] = details
            time.sleep(SLEEP_SEC)

        rows.append({
            "UniprotID": acc,
            "PDB": pdb,
            "details": details,
        })

out_df = pd.DataFrame(rows).drop_duplicates().reset_index(drop=True)
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
out_df.to_csv(OUT_CSV, index=False)

print("saved:", OUT_CSV)
print("row:", len(out_df))
print("UniprotID unique number:", out_df["UniprotID"].nunique())
print("PDB unique number:", out_df["PDB"].nunique())


### Extracting macromolecule entities from PDB 

In [ ]:
import pandas as pd
import requests
from pathlib import Path
from tqdm.auto import tqdm


IN_CSV = Path("GPCR_PDB2Uniprot.csv")
OUT_ENTITIES = Path("GPCR_PDB_macromolecules_entities.csv")
BASE_URL = "https://data.rcsb.org/rest/v1/core"


df = pd.read_csv(IN_CSV)
pdbs = df["PDB"].dropna().astype(str).str.strip().str.upper().unique()

session = requests.Session()
rows = []

for pdb in tqdm(pdbs, desc="Fetching Metadata"):
    entry_data = session.get(f"{BASE_URL}/entry/{pdb}").json()
    ent_ids = entry_data.get("rcsb_entry_container_identifiers", {}).get("polymer_entity_ids", [])

    for ent_id in ent_ids:
        ent_data = session.get(f"{BASE_URL}/polymer_entity/{pdb}/{ent_id}").json()

        desc = ent_data.get("rcsb_polymer_entity", {}).get("pdbx_description", "")
        chain = ent_data.get("entity_poly", {}).get("pdbx_strand_id", "")
        
        refs = ent_data.get("rcsb_polymer_entity_container_identifiers", {}).get("reference_sequence_identifiers", [])
        uniprots = sorted(set([
            r["database_accession"] for r in refs 
            if r.get("database_name") == "UniProt" and "database_accession" in r
        ]))

        rows.append({
            "PDB": pdb,
            "polymer_entity_id": ent_id,
            "chain_id": chain,
            "macromolecule_name": desc,
            "entity_uniprot_accessions": ";".join(uniprots)
        })

entities_df = pd.DataFrame(rows)
base_df = df[["PDB", "UniprotID"]].drop_duplicates()
base_df["PDB"] = base_df["PDB"].str.upper().str.strip()

final_df = entities_df.merge(base_df, on="PDB", how="left")
final_df.to_csv(OUT_ENTITIES, index=False)


/home/kjw/anaconda3/envs/hda2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PDB -> polymer entities: 100%|██████████| 2199/2199 [38:59<00:00,  1.06s/pdb] 


### Classification of PDB Structures Based on G-protein and Beta-arrestin Macromolecules

In [ ]:
import re
import pandas as pd
from pathlib import Path

IN_ENTITIES = Path("GPCR_PDB_macromolecules_entities.csv")
OUT_CSV = Path("Group/GPCR_PDB_group.csv")

df = pd.read_csv(IN_ENTITIES)


df["PDB"] = df["PDB"].astype(str).str.strip().str.upper()
df["UniprotID"] = df["UniprotID"].astype(str).str.strip().str.upper()

## Multiple GPCR complex PDB exception
multi_pdbs = set(
    df[["PDB","UniprotID"]].dropna().drop_duplicates()
      .groupby("PDB")["UniprotID"].nunique()
      .loc[lambda s: s >= 2].index
)

df = df.loc[~df["PDB"].isin(multi_pdbs)].copy()
df["chain_id"] = df.get("chain_id", "").fillna("").astype(str).str.strip()
df["macromolecule_name"] = df["macromolecule_name"].astype(str).str.replace(r"[‐-‒–—−]", "-", regex=True)

blob = df["macromolecule_name"].str.lower()

gprot_patterns = [
    r"\bg protein\b", r"\bheterotrimer", r"\bgi\b", r"\bgs\b", r"\bgq\b", r"\bgo\b",
    r"\bmini[- ]?g\b", r"\bminig\b",
    r"\bgnai", r"\bgnas", r"\bgnaq", r"\bgnao", r"\bgnb", r"\bgng",
    r"\bg alpha\b", r"\bg beta\b", r"\bg gamma\b",
    r"\bguanine nucleotide[- ]binding protein\b",
]
arr_patterns = [
    r"\bbeta[\s\-_]*arrestin(?:[\s\-_]*[12])?\b",
    r"\bβ[\s\-_]*arrestin(?:[\s\-_]*[12])?\b",
    r"\barrb[\s\-_]*1\b", r"\barrb[\s\-_]*2\b",
    r"\barrestin[\s\-_]*2\b", r"\barrestin[\s\-_]*3\b",
    r"\bbeta[\s\-_]*arrestin.*scfv",
]
exclude_arr_patterns = [
    r"\bs[\s\-_]*arrestin\b",
    r"\bvisual[\s\-_]*arrestin\b",
    r"\bsag\b",
    r"\bp10523\b",   # SAG(visual arrestin) UniProt
]

g_re = re.compile("|".join(gprot_patterns))
ex_re = re.compile("|".join(exclude_arr_patterns), flags=re.IGNORECASE)
a_re  = re.compile("|".join(arr_patterns), flags=re.IGNORECASE)

df["_is_g_"] = blob.apply(lambda s: bool(g_re.search(s)))
df["_is_a_"] = blob.apply(lambda s: (not ex_re.search(s)) and bool(a_re.search(s)))

def uniq_join(x):
    vals = []
    seen = set()
    for v in x:
        v = str(v).strip()
        if v and v not in seen:
            seen.add(v)
            vals.append(v)
    return ";".join(vals)

g_df = df[df["_is_g_"]].copy()
a_df = df[df["_is_a_"]].copy()

g_agg = (
    g_df.groupby("PDB", as_index=False)
        .agg(gprotein_chain_ids=("chain_id", uniq_join),
             gprotein_names=("macromolecule_name", uniq_join))
)

a_agg = (
    a_df.groupby("PDB", as_index=False)
        .agg(betaarrestin_chain_ids=("chain_id", uniq_join),
             betaarrestin_names=("macromolecule_name", uniq_join))
)

pdb_to_uniprot = (
    df[["PDB", "UniprotID"]]
    .dropna()
    .drop_duplicates()
    .groupby("PDB")["UniprotID"]
    .apply(lambda s: ";".join(pd.Series(s).astype(str).str.strip().replace("", pd.NA).dropna().drop_duplicates()))
    .reset_index()
)

all_pdb = (
    pd.DataFrame({"PDB": df["PDB"].dropna().drop_duplicates().sort_values()})
    .merge(pdb_to_uniprot, on="PDB", how="left")
)

out = all_pdb.merge(g_agg, on="PDB", how="left").merge(a_agg, on="PDB", how="left")

def classify(row):
    g = isinstance(row.get("gprotein_chain_ids"), str) and row["gprotein_chain_ids"].strip() != ""
    a = isinstance(row.get("betaarrestin_chain_ids"), str) and row["betaarrestin_chain_ids"].strip() != ""
    if g and not a:
        return "G-protein"
    if a and not g:
        return "Beta-arrestin"

out["Group"] = out.apply(classify, axis=1)
missing_pdbs = out.loc[out["Group"].isna() | (out["Group"].astype(str).str.strip() == ""), "PDB"]

if not missing_pdbs.empty:
    missing_df = df[df["PDB"].isin(missing_pdbs)]
    MISSING_CSV = Path("Group/GPCR_PDB_unclassified_entities.csv")
    missing_df.to_csv(MISSING_CSV, index=False)
    print(f"Saved (PDBs: {len(missing_pdbs)})")

out = out[out["Group"].notna() & (out["Group"].astype(str).str.strip() != "")]
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
out.to_csv(OUT_CSV, index=False)
print("saved:", OUT_CSV, "rows:", len(out))
print(out['Group'].value_counts())




saved: /home/kjw/GPCRbias/Data/GPCR/Group/GPCR_PDB_group.csv rows: 1200


### Sequence alignment and PDB chain extraction

In [ ]:
import os
import requests
import pandas as pd
import warnings
from Bio.PDB import MMCIFParser
from Bio import pairwise2
from Bio.Align import substitution_matrices
from tqdm import tqdm

IN_CSV = "Group/GPCR_PDB_group.csv"
OUT_FILE = 'Group/GPCR_PDB_chain.csv'
FASTA_DIR = "uniprot_fasta"
CIF_DIR = "pdb_cif"


os.makedirs(CIF_DIR, exist_ok=True)
os.makedirs(FASTA_DIR, exist_ok=True)
warnings.filterwarnings("ignore")

AA_MAP = {
    'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D', 'CYS': 'C',
    'GLN': 'Q', 'GLU': 'E', 'GLY': 'G', 'HIS': 'H', 'ILE': 'I',
    'LEU': 'L', 'LYS': 'K', 'MET': 'M', 'PHE': 'F', 'PRO': 'P',
    'SER': 'S', 'THR': 'T', 'TRP': 'W', 'TYR': 'Y', 'VAL': 'V',
    'MSE': 'M', 'PTR': 'Y', 'TPO': 'T', 'SEP': 'S'
}

def to_1letter(resname):
    return AA_MAP.get(resname, 'X')

def download_cif(pdb_id):
    path = os.path.join(CIF_DIR, f"{pdb_id}.cif")
    if os.path.exists(path) and os.path.getsize(path) > 1000:
        return path
    
    url = f"https://files.rcsb.org/download/{pdb_id}.cif"
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    with open(path, "wb") as f:
        f.write(r.content)
    return path

def fetch_uniprot_seq(uniprot_id):
    if not uniprot_id or pd.isna(uniprot_id): return None
    
    path = os.path.join(FASTA_DIR, f"{uniprot_id}.fasta")
    
    if not (os.path.exists(path) and os.path.getsize(path) > 0):
        url = f"https://www.uniprot.org/uniprot/{uniprot_id}.fasta"
        r = requests.get(url, timeout=10)
        r.raise_for_status()
        with open(path, "w") as f:
            f.write(r.text)
            
    # 파일 읽어서 서열 반환 (헤더 제외)
    with open(path, "r") as f:
        lines = f.readlines()
        
    return "".join([line.strip() for line in lines if not line.startswith(">")])

def extract_chain_seq(structure, chain_id):
    if chain_id not in structure[0]: return None
    chain = structure[0][chain_id]
    residues = [res for res in chain.get_residues() if 'CA' in res]
    seq = ""
    for res in residues:
        seq += to_1letter(res.get_resname())
    return seq

def compute_alignment(uniprot_seq, pdb_seq, matrix):
    if not uniprot_seq or not pdb_seq: return None

    alignments = pairwise2.align.localds(uniprot_seq, pdb_seq, matrix, -10, -0.5)
    
    if not alignments: return None

    best = alignments[0]
    seqA, seqB = best.seqA, best.seqB
    
    matches = sum(1 for a, b in zip(seqA, seqB) if a == b and a != '-')
    aligned_len = sum(1 for a, b in zip(seqA, seqB) if a != '-' or b != '-')
    is_pass = (matches / len(uniprot_seq) >= 0.7) & (matches / aligned_len >= 0.7) & (matches / len(pdb_seq) >= 1.0)
        

    return {
        'score': best.score,
        'identity': matches / aligned_len if aligned_len else 0,
        'coverageUniProt': matches / len(uniprot_seq),
        'coverageChain': matches / len(pdb_seq),
        'chain_length': len(pdb_seq),
        'Pass' : is_pass
    }

if __name__ == "__main__":
    df = pd.read_csv(IN_CSV)
    unique_pairs = df[['PDB', 'UniprotID']].dropna().drop_duplicates()
    
    parser = MMCIFParser(QUIET=True)
    
    blosum62 = substitution_matrices.load("BLOSUM62")
    
    results = []
    pbar = tqdm(unique_pairs.iterrows(), total=unique_pairs.shape[0], desc="Starting")
    
    for i, (_, row) in enumerate(unique_pairs.iterrows()):
        pdb_id = row['PDB'].strip().upper()
        uniprot_id = str(row['UniprotID']).strip()
        
        cif_path = download_cif(pdb_id)
        if not cif_path: continue

        uni_seq = fetch_uniprot_seq(uniprot_id)
        if not uni_seq: continue

        try:
            structure = parser.get_structure(pdb_id, cif_path)
        except Exception:
            continue
        
        for chain in structure[0]:
            chain_id = chain.id
            pdb_seq = extract_chain_seq(structure, chain_id)
            
            if not pdb_seq or len(pdb_seq) < 10: continue
            
            metrics = compute_alignment(uni_seq, pdb_seq, blosum62)
            if metrics and metrics['score'] > 0:
                results.append({
                    'PDB': pdb_id,
                    'UniprotID': uniprot_id,
                    'chain_id': chain_id,
                    **metrics
                })
        
        if (i + 1) % 10 == 0:
            print(f"{i + 1}/{len(unique_pairs)}")

    res_df = pd.DataFrame(results)
    
    final_df = pd.merge(df, res_df, on=['PDB', 'UniprotID'], how='right')
    
    final_df.to_csv(OUT_FILE, index=False)

### Group Data Table

In [20]:
import pandas as pd

CHAIN_CSV = 'Group/GPCR_PDB_chain.csv'
GROUP_CSV = 'Group/GPCR_PDB_group.csv'

chain_df = pd.read_csv(CHAIN_CSV)
group_df = pd.read_csv(GROUP_CSV)

passed_pdbs = set(chain_df.loc[chain_df['Pass'] == True, 'PDB'])

valid_group_df = group_df[group_df['PDB'].isin(passed_pdbs)]

total_counts = group_df['Group'].value_counts().rename("Total_PDBs")
valid_counts = valid_group_df['Group'].value_counts().rename("Passed_PDBs")

summary_df = pd.concat([total_counts, valid_counts], axis=1).fillna(0).astype(int)

print(summary_df)

               Total_PDBs  Passed_PDBs
Group                                 
G-protein            1141          706
Beta-arrestin          58           19


In [21]:
import pandas as pd
from pathlib import Path

csv_path = Path("/home/kjw/GPCRbias/Data/GPCR/Group/GPCR_PDB_chain.csv")
df = pd.read_csv(csv_path)

# 1. Pass가 True인 데이터만 필터링
pass_df = df[df['Pass'] == True].copy()

# 2. PDB별 고유 chain_id 개수 카운트
chain_counts = pass_df.groupby('PDB')['chain_id'].nunique()

# 3. chain이 2개 이상인 PDB 식별
multi_chain_pdbs = chain_counts[chain_counts > 1].index

# 4. 해당 PDB들의 상세 정보 출력
result = pass_df[pass_df['PDB'].isin(multi_chain_pdbs)].sort_values(['PDB', 'chain_id'])

print(f"중복 체인을 가진 PDB 수: {len(multi_chain_pdbs)}")
print("-" * 30)
print(result[['PDB', 'chain_id', 'UniprotID', 'Pass']])

중복 체인을 가진 PDB 수: 31
------------------------------
       PDB chain_id UniprotID  Pass
5094  7E9G        R    Q14416  True
5095  7E9G        S    Q14416  True
3040  7E9H        R    Q14833  True
3041  7E9H        S    Q14833  True
2392  7MTS        A    Q14416  True
...    ...      ...       ...   ...
989   9M8V        E    P46089  True
649   9MB9        D    O00222  True
650   9MB9        R    O00222  True
3000  9MBA        D    O00222  True
3001  9MBA        R    O00222  True

[62 rows x 4 columns]
